In [1]:
!pip install arxiv

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6061 sha256=739319ba5945ab462dbc1adad287be9c98e29bab3c6f0b6915cd7fc820b5aa47
  Stored in directory: c:\users\1\appdata\local\pip\cache\wheels\f0\69\93\a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k


In [3]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [15]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [6]:
tool.name

'wikipedia'

In [8]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
# from langchain_community.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter


loader = WebBaseLoader("https://python.langchain.com/docs/introduction/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=200).split_documents(docs)
vector_db = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector_db.as_retriever()
retriever



VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002ECA074F700>, search_kwargs={})

In [9]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, "introduce_search", "Search information about Tutorial. For any questions about tutorial whis introduse you must use this tool" )

In [10]:
retriever_tool.name

'introduce_search'

In [11]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

In [13]:
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1,
                                 doc_content_chars_max=1000)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [16]:
tools = [wiki, arxiv, retriever_tool]

In [17]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\User\\Evheniia\\test tasks\\fastapi_streamlit\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=100)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=1000)),
 Tool(name='introduce_search', description='Search information about Tutorial. For any questions about tutorial whis introduse you must use this tool', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000002EC9001D900>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vec

In [22]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [24]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages 

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [25]:
from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)

In [26]:
from langchain.agents import AgentExecutor
agent_exe = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_exe

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [29]:
agent_exe.invoke({"input": "Tell me about introduse page"})



> Entering new AgentExecutor chain...

Invoking: `introduce_search` with `{'query': 'introduse'}`


Expression Language (LCEL)MessagesMultimodalityOutput parsersPrompt TemplatesRetrieval augmented generation (RAG)RetrievalRetrieversRunnable interfaceStreamingStructured outputsTestingString-in,

developer's guide for guidelines on contributing and help getting your dev environment set up.Edit this pageWas this page helpful?NextTutorialsTutorialsHow-to guidesConceptual guideIntegrationsAPI

HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt

guideIntegrationsAPI referenceEcosystem🦜🛠️ LangSmith🦜🕸️ LangGraphAdditional resourcesVersionsSecurityContributingCommunityTwitterGitHubOrganizationPythonJS/TSMoreHomepageBlogYouTubeCopyright © 2024The "introduse" page seems to be related to a developer's guide for guidelines on contributing and setting up the de

{'input': 'Tell me about introduse page',
 'output': 'The "introduse" page seems to be related to a developer\'s guide for guidelines on contributing and setting up the development environment. It also includes information on tutorials, how-to guides, conceptual guides, integrations, API references, and additional resources. The page may provide tutorials on building a Question Answering application over a Graph Database and building a simple LLM application with chat models and prompt guides. It also mentions versions, security, contributing guidelines, community links, and copyright information.'}